# Parsing Data
In this notebook, let's explore how to leverage generative AI to build and consume a knowledge graph in Neo4j.

This notebook parses Form-13 data from SEC EDGAR. This is partially structured data, a mix of text and XML.  Instead of spending our time writing a bespoke parser to extract data from these files and load into Neo4j, we can prompt a Large Language Model (LLM) to do this for us automatically.  We will then also use the LLM to generate Cypher statements to load the extracted data into a Neo4j graph.

## Setup
First, let's install the libraries we're going to need for this lab and the following notebook dependent labs.  We'll also want to reboot the kernel once done.  To do that, go to the "Kernel" menu and click "Restart Kernel and Clear All Outputs."  That will get rid of everything the install statements printed, leaving us with a cleaner notebook to work with.

In [ ]:
# Used in Labs 5, 6 and 7
%pip install --user openai
%pip install --user graphdatascience
%pip install --user tqdm
%pip install --user IProgress
%pip install --user ipywidgets

Now restart the kernel. That will allow the Python evironment to import the new packages.

## Prompt Definition
We will extract knowledge adhering to the same schema we used previously.  To teach the LLM about the schema, we will use a series of prompts.  Each prompt is focused on only one task, extracting a specific entity:

1. Manager Information
2. Filing Information

In [ ]:
mgr_info_tpl = """From the text below, extract the following as json. Do not miss any of these information.
* The tags mentioned below may or may not namespaced. So extract accordingly. Eg: <ns1:tag> is equal to <tag>
* "managerName" - The name from the <name> tag under <filingManager> tag
* "street1" - The manager's street1 address from the <com:street1> tag under <address> tag
* "street2" - The manager's street2 address from the <com:street2> tag under <address> tag
* "city" - The manager's city address from the <com:city> tag under <address> tag
* "stateOrCounty" - The manager's stateOrCounty address from the <com:stateOrCountry> tag under <address> tag
* "zipCode" - The manager's zipCode from the <com:zipCode> tag under <address> tag
* "reportCalendarOrQuarter" - The reportCalendarOrQuarter from the <reportCalendarOrQuarter> tag under <address> tag
* Just return me the JSON enclosed by 3 backticks. No other text in the response

Text:
$ctext
"""

In [ ]:
filing_info_tpl = """The text below contains a list of investments. Each instance of <infoTable> tag represents a unique investment. 
For each investment, please extract the below variables into json then combine into a list enclosed by 3 backticks. Please use the quoted names below while doing this
* "cusip" - The cusip from the <cusip> tag under <infoTable> tag
* "companyName" - The name under the <nameOfIssuer> tag.
* "value" - The value from the <value> tag under <infoTable> tag. Return as a number. 
* "shares" - The sshPrnamt from the <sshPrnamt> tag under <infoTable> tag. Return as a number. 
* "sshPrnamtType" - The sshPrnamtType from the <sshPrnamtType> tag under <infoTable> tag
* "investmentDiscretion" - The investmentDiscretion from the <investmentDiscretion> tag under <infoTable> tag
* "votingSole" - The votingSole from the <votingSole> tag under <infoTable> tag
* "votingShared" - The votingShared from the <votingShared> tag under <infoTable> tag
* "votingNone" - The votingNone from the <votingNone> tag under <infoTable> tag

Output format:
* DO NOT output XML tags in the response. The output should be a valid JSON list enclosed by 3 backticks

Text:
$ctext
"""

## Functions for Using LLMs
Let's create some helper function to talk to the LLM with our prompt and text input. 

The [Azure OpenAI documentation](https://learn.microsoft.com/en-us/azure/ai-services/openai/) describes the available foundation models.  We will use the GPT-4 base model. Go to Azure OpenAI and copy-paste the following information from the `Keys and Endpoints` page under `Resource Management` section in the left  

In [ ]:
# Change this!
 
API_ENDPOINT = 'https://azure-oai-lab.openai.azure.com/'
API_VERSION = '2024-02-01'
API_KEY = '916bf018923d43b086c1629b0c80e6ce'

In [ ]:
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = API_ENDPOINT, 
  api_key = API_KEY,  
  api_version = API_VERSION
)

In [ ]:
import traceback
def call_language_model(system_prompt, prompt):
    try:
        completion = client.chat.completions.create(
            model="gpt-4o",
            max_tokens=4096,
            temperature=0,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt},
            ]
        )
        return completion.choices[0].message.content
    except Exception as e:
        traceback.format_exc()
        print(e)

In [ ]:
# Wrapper for entity extraction and parsing
def extract_entities_relationships(prompt):
    try:
        res = call_language_model('You are an NLP expert who can extract Entities and Relationships from text', prompt)
        return res
    except Exception as e:
        print(e)

In [ ]:
import re
import numpy as np

# splitting function for chunking up filing information to avoid hitting LLM token limits
def split_filing_info(s, chunk_size=10):
    pattern = '(</(\w+:)?infoTable>)'
    splitter = re.findall(pattern, s)[0][0]
    _parts = s.split(splitter)
    if len(_parts) > chunk_size:
        chunks_of_list = np.array_split(_parts, len(_parts)/chunk_size) # max 20 filings per part
        chunks_of_str = map(lambda x: splitter.join(x)+splitter, chunks_of_list)
        l = list(chunks_of_str)
        if len(l) > 0:
            l[len(l)-1] = re.sub(f'{splitter}$', '', l[len(l)-1])
        return l
    else:
        return [s]

## Test Example for Parsing
Let's start with one Form 13 file to see how we can parse it with Generative AI.

In [ ]:
from azure.storage.blob import BlobServiceClient

blob_service_client = BlobServiceClient(account_url="https://neo4jdataset.blob.core.windows.net/")

blob_client = blob_service_client.get_blob_client(container='form13-raw', blob="raw_2023-07-18_archives_edgar_data_712011_0000905729-23-000116.txt")

# encoding param is necessary for readall() to return str, otherwise it returns bytes
downloader = blob_client.download_blob(max_concurrency=1, encoding='UTF-8')
inp_text = downloader.readall()

We can take a look at the file.  Note that it is an oddball mix of XML, delimeted and fixed spacing formatting that no standard parser could make sense of.

In [ ]:
print(inp_text[:1500])

We can split data into manager and filing info pieces using `<XML>` tags

In [ ]:
contents = inp_text.split('<XML>')
manager_info = contents[1].split('</XML>')[0].strip()
filing_info = contents[2].split('</XML>')[0].strip()

## Parsing Manager Information

In [ ]:
from string import Template

prompt = Template(mgr_info_tpl).substitute(ctext=manager_info)
print(prompt)

In [ ]:
import json

# Use LLM to parse out manager information
manager_data = json.loads(extract_entities_relationships(prompt).split('```')[1].strip('json'))
manager_data

## Parse Filing Information
We will parse filing info in a similar manner to manager information. Because the filings include a list of many entries however, we will want to split the input into chunks so as not to exceed input or output token limits. 

In [ ]:
filing_info_chunks = split_filing_info(filing_info)
len(filing_info_chunks)

In [ ]:
%%time
prompt = Template(filing_info_tpl).substitute(ctext=filing_info_chunks[0])
response = extract_entities_relationships(prompt)
print(response)

## Test Example

Let's walk through the steps to do this with just the 1 form above first, then we can move on to parsing and ingesting multiple form13s

To start we can run the LLM parsing over all the filing info from the form and then combine the resulting JSON into a list conducive for Neo4j loading. This will take approximately 2 minutes to complete

In [ ]:
%%time

filings_list = []
for filing_info_chunk in filing_info_chunks:
    prompt = Template(filing_info_tpl).substitute(ctext=filing_info_chunk)
    response = extract_entities_relationships(prompt)
    if '```' in response:
        response = response.split('```')[1].strip('json')
    filings_list.extend(json.loads(response))

for item in filings_list:
    item['managerName'] = manager_data['managerName']
    item['reportCalendarOrQuarter'] = manager_data['reportCalendarOrQuarter']
filings_list[:2]

In [ ]:
len(filings_list)

## Establish Neo4j Connection

Replace the following Neo4j credentials with the instance you created earlier

In [ ]:
# Username is neo4j by default
NEO4J_USERNAME = 'neo4j'

# You will need to change these to match your credentials
NEO4J_URI = 'neo4j://vm0.node-3hvyxlb2vkjy2.eastus.cloudapp.azure.com'
NEO4J_PASSWORD = 'Foo12345678!'

In [ ]:
from graphdatascience import GraphDataScience

gds = GraphDataScience(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD),
    aura_ds=False
)
gds.set_database('neo4j')

Before loading, we should create node key constraints for nodes.  This acts as a unique id and an index and is necessary for fast, efficient queries.  In general, if you notice ingestion is super slow (and getting slower) with Neo4j, double-check that you created indexes.  For this small sample, it won't matter, but it will undoubtedly impact as we ingest more data. 

In [ ]:
gds.run_cypher('CREATE CONSTRAINT unique_manager IF NOT EXISTS FOR (n:Manager) REQUIRE (n.managerName) IS NODE KEY')
gds.run_cypher('CREATE CONSTRAINT unique_company_id IF NOT EXISTS FOR (n:Company) REQUIRE (n.cusip) IS NODE KEY')

To merge the data, we can use parameterized Cypher queries.  Basically, we will send filings in batches (in this sample case, just one batch) for each node and relationship type and insert them as parameters in the query.

In [ ]:
# Merge company nodes
gds.run_cypher('''
UNWIND $records AS record
MERGE (c:Company {cusip: record.cusip})
SET c.companyName = record.companyName
RETURN count(c) AS company_node_merge_count
''', params={'records':filings_list})

In [ ]:
# Merge manager node
gds.run_cypher('''
MERGE (m:Manager {managerName: $name})
RETURN count(m) AS manager_node_merge_count
''', params={'name':manager_data['managerName']})

In [ ]:
# Merge owns Relationship
gds.run_cypher('''
UNWIND $records AS record
MATCH (m:Manager {managerName: record.managerName})
MATCH (c:Company {cusip: record.cusip})
MERGE(m)-[r:OWNS]->(c)
SET r.reportCalendarOrQuarter = record.reportCalendarOrQuarter,
    r.value = record.value,
    r.shares = record.shares
RETURN count(r) AS owns_relationship_merge_count
''', params={'records':filings_list})

## Ingest Multiple Form 13 Files
We will make a pipeline using the methods above.  In this case we will take a two-step approach, first parse all the data, then chunk that data and ingest into Neo4j.

For purposes of this lab we will just use a few form13 files.

In [ ]:
sample_file_names = [
   'raw_2023-07-18_archives_edgar_data_1108893_0001108893-23-000005.txt',
   'raw_2023-07-18_archives_edgar_data_1488921_0001085146-23-002736.txt',
   'raw_2023-07-18_archives_edgar_data_1163165_0001104659-23-081874.txt',
   'raw_2023-07-18_archives_edgar_data_1567459_0000950123-23-006124.txt'
]

In [ ]:
# Helper function for getting filing info
def get_manager_and_filing_info(raw_txt):
    contents = raw_txt.split('<XML>')
    manager_info = contents[1].split('</XML>')[0].strip()
    filing_info = contents[2].split('</XML>')[0].strip()
    
    return manager_info, filing_info

This parsing will probably take around 2-4 mins

In [ ]:
%%time

blob_service_client = BlobServiceClient(account_url="https://neo4jdataset.blob.core.windows.net/")
def read_file(filename):
    blob_client = blob_service_client.get_blob_client(container='form13-raw', blob=filename)
    downloader = blob_client.download_blob(max_concurrency=1, encoding='UTF-8')
    return downloader.readall()

print(f'=== Parsing {len(sample_file_names)} Form 13 Files ===')

filings_list = []
manager_list = []

for file_name in sample_file_names:
    
    print(f'--- parsing {file_name} ---')
    try:
        # Get raw form13 file
        print('getting file text from Blob Storage....')
        raw_text = read_file(file_name)

        # Get raw manager and filing info from file
        print('getting file contents...')
        manager_info, filing_info = get_manager_and_filing_info(raw_text)

        # Parse manager info into dict using LLM
        print('Parsing submission and manager info...')
        mng_prompt = Template(mgr_info_tpl).substitute(ctext=manager_info)
        mng_response = extract_entities_relationships(mng_prompt).split('```')[1].strip('json')
        manager_data = json.loads(mng_response)
        manager_list.append({'managerName': manager_data['managerName']})

        # Parse filing info into list of dicts using LLM
        print('Parsing filing info...')
        tmp_filing_list = []
        for filing_info_chunk in split_filing_info(filing_info):
            filing_prompt = Template(filing_info_tpl).substitute(ctext=filing_info_chunk)
            filing_response = extract_entities_relationships(filing_prompt)
            if '```' in filing_response:
                filing_response = filing_response.split('```')[1].strip('json')
            tmp_filing_list.extend(json.loads(filing_response))
        for item in tmp_filing_list: #Add information from manager_info to enable OWNS relationship loading
            item['managerName'] = manager_data['managerName']
            item['reportCalendarOrQuarter'] = manager_data['reportCalendarOrQuarter']
        filings_list.extend(tmp_filing_list)
    except Exception as e:
        print(e)
        raise e


Now we can merge the manager nodes

In [ ]:
# Merge manager nodes
gds.run_cypher('''
UNWIND $records AS record
MERGE (m:Manager {managerName: record.managerName})
RETURN count(m) AS manager_node_merge_count
''', params={'records':manager_list})

For filings lets check ther length of the list

In [ ]:
len(filings_list)

While we should not need chunking for this example, below is an example of how to chunk up a parameterized function for loading in case you need to scale up. 

In [ ]:
# As the dataset gets bigger we will want to chunk up the filings we send to Neo4j
def chunks(xs, n=10_000):
    n = max(1, n)
    return [xs[i:i + n] for i in range(0, len(xs), n)]

In [ ]:
# Merge company nodes
for d in chunks(filings_list):
    res = gds.run_cypher('''
    UNWIND $records AS record
    MERGE (c:Company {cusip: record.cusip})
    SET c.companyName = record.companyName
    RETURN count(c) AS company_node_merge_count
    ''', params={'records':d})
    print(res)

In [ ]:
# Merge owns Relationships
for d in chunks(filings_list):
    res = gds.run_cypher('''
    UNWIND $records AS record
    MATCH (m:Manager {managerName: record.managerName})
    MATCH (c:Company {cusip: record.cusip})
    MERGE(m)-[r:OWNS]->(c)
    SET r.reportCalendarOrQuarter = record.reportCalendarOrQuarter,
        r.value = record.value,
        r.shares = record.shares
    RETURN count(r) AS owns_relationship_merge_count
    ''', params={'records':d})
    print(res)

This type of workflow can be applied to other unstructured data to parse entities and relationships with language models and load them into a Neo4j knowledge graph. 